In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib
import matplotlib.pyplot as plt # for plotting
import plotly.offline as py
py.init_notebook_mode(connected=True)
import seaborn as sns # for making plots with seaborn
color = sns.color_palette()
import plotly.graph_objs as go
import plotly.offline as offline
offline.init_notebook_mode()
import plotly.tools as tls
from numpy import array
from matplotlib import cm

In [ ]:
#path = r'Chronic_illness'

In [ ]:
path = r'C:\Users\tiffk\Chronic_Illness'

In [ ]:
# Importing dataset
df_checked = pd.read_csv(os.path.join(path, 'Data', 'chronic_checked.csv'), low_memory=False, index_col = False)

In [ ]:
df_checked['checkin_date'] = pd.to_datetime(df_checked['checkin_date'])  # Convert 'checkin_date' to datetime format
df_checked['entry_id'] = df_checked['user_id'].astype(str) + '_' + df_checked['checkin_date'].dt.strftime('%Y-%m-%d')

In [ ]:
# To keep track of entries during merging and track user based and weather and date based factors, create an identifier with 'user_id' and 'checkin_date' columns
df_checked['entry_id'] = df_checked['user_id'] + '_' + df_checked['checkin_date'].dt.strftime('%Y-%m-%d')

In [ ]:
df_checked['entry_id'] = df_checked['user_id'].astype(str) + '_' + df_checked['checkin_date'].dt.strftime('%Y-%m-%d')

In [ ]:
df_checked.info()

In [ ]:
s_max = df_checked[df_checked.trackable_name=="temperature_max"].trackable_value
s_min = df_checked[df_checked.trackable_name=="temperature_min"].trackable_value

In [ ]:
max_temp = pd.to_numeric(s_max, errors='coerce')
min_temp = pd.to_numeric(s_min, errors='coerce')

In [ ]:
df_condition = df_checked[df_checked.trackable_type=="Condition"]

In [ ]:
# Pivot the DataFrame to create new columns from the values under "trackable_name"
new_df_condition = df_condition.pivot_table(index='user_id', columns='trackable_type', values='trackable_name', aggfunc='first').reset_index()

In [ ]:
df_checked[df_checked.trackable_type=="Weather"].trackable_name.value_counts()

In [ ]:
df_weather=df_checked[df_checked.trackable_type=="Weather"]


In [ ]:
df_weather.head()

In [ ]:

# Pivot the DataFrame to create new columns from the values under "trackable_name"
new_df_weather = df_weather.pivot_table(index='user_id', columns='trackable_name', values='trackable_value', aggfunc='first').reset_index()

# This will create new columns in the DataFrame based on the unique values in "trackable_name", and the values will be populated from "trackable_value"

In [ ]:
new_df_weather.head(10)

In [ ]:
new_df_weather.describe()

In [ ]:
unique_trackable_types = df_checked['trackable_type'].unique()
print(unique_trackable_types)

In [ ]:
df_symptom=df_checked[df_checked.trackable_type=="Symptom"]
df_treatment=df_checked[df_checked.trackable_type=="Treatment"]
df_tag=df_checked[df_checked.trackable_type=="Tag"]
df_food=df_checked[df_checked.trackable_type=="Food"]
df_HBI=df_checked[df_checked.trackable_type=="HBI"]

In [ ]:
df_symptom.info()

In [ ]:
df_checked.info()

In [ ]:
df_checked.info()

In [ ]:
df_weather.info()

In [ ]:
new_df_weather['temperature_change'] = new_df_weather['temperature_max'] - new_df_weather['temperature_min']

In [ ]:
new_df_weather['humidity'] = pd.to_numeric(new_df_weather['humidity'])


In [ ]:
new_df_weather['temperature_max'] = pd.to_numeric(new_df_weather['temperature_max'])
new_df_weather['temperature_min'] = pd.to_numeric(new_df_weather['temperature_min'])

In [ ]:
new_df_weather['precip_intensity'] = pd.to_numeric(new_df_weather['precip_intensity'])
new_df_weather['pressure'] = pd.to_numeric(new_df_weather['pressure'])

In [ ]:
new_df_weather.info()

In [ ]:
df_checked['age'] = pd.to_numeric(df_checked['age'])


In [ ]:
merged_df = df_condition.merge(df_treatment, on='entry_id', how='outer')

In [ ]:
merged_df.head(5)

In [ ]:
df_checked.head()

In [ ]:
# Assuming df_checked is your DataFrame
# Create a new column 'condition' and set default value as None
df_checked['condition'] = None

# Set the values from 'trackable_name' where 'trackable_type' is 'Condition'
df_checked.loc[df_checked['trackable_type'] == 'Condition', 'condition'] = df_checked.loc[df_checked['trackable_type'] == 'Condition', 'trackable_name']

In [ ]:
# Create a new column in df_checked 'treatment' and set default value as None
df_checked['treatment'] = None

# Set the values from 'trackable_name' where 'trackable_type' is 'Condition'
df_checked.loc[df_checked['trackable_type'] == 'Treatment', 'treatment'] = df_checked.loc[df_checked['trackable_type'] == 'Treatment', 'trackable_name']

In [ ]:
unique_trackable_types = df_checked['trackable_type'].unique()
print(unique_trackable_types)

In [ ]:
# Create a new column 'treatment' in df_checked and set default value as None
df_checked['food'] = None

# Set the values from 'trackable_name' where 'trackable_type' is 'Food'
df_checked.loc[df_checked['trackable_type'] == 'Food', 'food'] = df_checked.loc[df_checked['trackable_type'] == 'Food', 'food']

In [ ]:
# In df_checked create a new column 'pain' and set default value as 0
df_checked['pain'] = '0'

# Set the values from 'Condition' where 'pain' is found
df_checked.loc[df_checked['trackable_type'] == 'Condition', 'condition'] = df_checked.loc[df_checked['trackable_type'] == 'Food', 'food']

In [ ]:
# Creates a new column 'Pain' in df_checked and set the default value as 0
df_checked['Pain'] = 0

# Set the value to 1 where the word 'pain' is found in the 'Condition' column
df_checked.loc[df_checked['condition'].str.contains('pain', case=False, na=False), 'Pain'] = 1

In [ ]:
df_checked.head()

In [ ]:
# Create a new column 'Pain_Severity' in df_checked by multiplying the 'Pain' column with 'trackable_value' where 'trackable_type' is 'Condition'
df_checked['Pain_Severity'] = df_checked['Pain'] * df_checked.loc[df_checked['trackable_type'] == 'Condition', 'trackable_value']

In [ ]:
df_checked.info()

In [ ]:
df_weather.head()

In [ ]:
# Exporting df_checked as "chronic_transformed.csv"
df_checked.to_csv(os.path.join(path, 'Data', 'chronic_transformed.csv'))

In [ ]:
# Create a new column 'pressure' in df_weather with values from 'trackable_value' where 'trackable_name' is 'pressure'
df_weather['pressure'] = df_weather.loc[df_weather['trackable_name'] == 'pressure', 'trackable_value']

In [ ]:
df_weather.describe()

In [ ]:
# Create a new column 'precipitation' in df_weather with values from 'trackable_value' where 'trackable_name' is 'precip_intensity'
df_weather['precipitation'] = df_weather.loc[df_weather['trackable_name'] == 'precip_intensity', 'trackable_value']

In [ ]:
# Create a new column 'precipitation' in df_weather with values from 'trackable_value' where 'trackable_name' is 'precip_intensity'
df_weather['precipitation'] = df_weather.loc[df_weather['trackable_name'] == 'precip_intensity', 'trackable_value']

In [ ]:
# Create a new column 'max_temp' in df_weather with values from 'trackable_value' where 'trackable_name' is 'temperature_max'
df_weather['max_temp'] = df_weather.loc[df_weather['trackable_name'] == 'temperature_max', 'trackable_value']

In [ ]:
# Create a new column 'min_temp' in df_weather with values from 'trackable_value' where 'trackable_name' is 'temperature_min'
df_weather['min_temp'] = df_weather.loc[df_weather['trackable_name'] == 'temperature_min', 'trackable_value']

In [ ]:
new_df_weather.info()

In [ ]:

# Assuming df_weather is your DataFrame
df_weather['temp_change'] = df_weather['max_temp'] - df_weather['min_temp']

In [ ]:

# Assuming df_weather is your DataFrame and cols_to_convert is a list of column names to convert to numeric
cols_to_convert = ['column1', 'column2', 'column3']  # Replace with your actual column names

df_weather[cols_to_convert] = df_weather[cols_to_convert].apply(pd.to_numeric, errors='coerce')

In [ ]:
# Merging df_checked and df_weather DataFrames
merged_df = df_checked.merge(df_weather, on='entry_id', how='left')

In [ ]:
# Exporting df_checked as "chronic_transformed.csv"
df_weather.to_csv(os.path.join(path, 'Data', 'weather_combined.csv'))

In [ ]:
df_weather.head()

In [ ]:
merged_df

In [ ]:
# Exporting merged_df as "chronic_merged.csv"
merged_df.to_csv(os.path.join(path, 'Data', 'chronic_merged.csv'))

In [ ]:
merged_df.info()

In [ ]:
merged_df.describe()

In [ ]:
# Selecting only necessary columns from df_checked
df_checked_reduced = df_checked[['entry_id', 'age', 'sex', 'country', 'checkin_date', 'condition', 'treatment', 'food', 'Pain','Pain_Severity']]

# Selecting only necessary columns from df_weather
df_weather_reduced = df_weather[['entry_id', 'pressure', 'precipitation', 'max_temp', 'min_temp']]

In [ ]:
# Merging df_checked and df_weather DataFrames
remerged_df = df_checked_reduced.merge(df_weather_reduced, on='entry_id', how='left')

In [ ]:
# Exporting remerged_df as "chronic_merged.csv"
remerged_df.to_csv(os.path.join(path, 'Data', 'chronic_merged_num.csv'))

In [ ]:

# Replace empty strings with NaN and then convert the columns to float
remerged_df[['age', 'Pain', 'Pain_Severity', 'pressure', 'precipitation', 'max_temp', 'min_temp']] = remerged_df[['age', 'Pain', 'Pain_Severity', 'pressure', 'precipitation', 'max_temp', 'min_temp']].replace('', float('nan')).astype(float)

In [ ]:
# Calculate the correlation matrix
corr_matrix = remerged_df.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
# Remove non-numeric columns from the dataframe before calculating the correlation matrix
numeric_df = remerged_df.select_dtypes(include=['float64', 'int64'])

# Calculate the correlation matrix
corr_matrix = numeric_df.corr()

# Create a heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()

In [ ]:
numerical_columns = remerged_df(include=['int', 'float'])

In [ ]:
#Create a correlation matrix using pandas
#Select only the numerical columns from the DataFrame to avoid errors
numerical_columns = remerged_df.select_dtypes(include=['int', 'float'])

#The index ('rank') is one of the numerical columns, so we will drop it

if 'age' in numerical_columns.columns:
    numerical_columns = numerical_columns.drop(columns=['age', 'max_temp', 'min_temp'])

#Now, we create the correlation matrix with the appropriate data
numerical_columns.corr()

In [ ]:
remerged_df['temp_change'] = remerged_df['max_temp'] - remerged_df['min_temp']

In [ ]:
# Create a new DataFrame with selected columns
selected_columns_df = remerged_df[['Pain', 'Pain_Severity', 'pressure', 'precipitation', 'temp_change']]